In [6]:
from fscve_step1_db import AnalysisState,FscveStep1DbHelper
dbhelper = FscveStep1DbHelper("nm", "fca", 2, False)
session = dbhelper.session
expid = dbhelper.experiment_id
benchmark = dbhelper.benchmark
fc = dbhelper.fc
# query = session.query(AnalysisState.analysis_id,AnalysisState.discovery_id).first()
# print(query)
print(f"{benchmark}---{fc}-----{expid}")

nm---fca-----2


In [7]:
from fscve_step1_db import  TestCase
import pandas as pd
# calculate the total number of uniq crashes for step7 metric1
queryca = session.query(TestCase).filter(TestCase.test_case_type_id == 2).all()
dict_value_benchmark = [f"{benchmark}"  for result in queryca]
dict_value_fc = [f"{fc}"  for res in queryca]
dict_value_expid = [f"{expid}"  for res in queryca]
dict_value_testcase_hash = [result.hash  for result in queryca]

dictca = {"benchmark":dict_value_benchmark, "fuzzer_combination":dict_value_fc, "expid":dict_value_expid,"testcase_hash":dict_value_testcase_hash}
df_unique_crashes = pd.DataFrame(dictca)
#print(df_unique_crashes)
df_unique_crashes.to_csv(f"csv/fscve_unique_crashes_step7metric1_{benchmark}_{fc}_{expid}.csv")
# print(df_bug_count)

In [8]:
from fscve_step1_db import  CrashAnalysis
import pandas as pd
# calculate the total number of uniq real bugs
queryca = session.query(CrashAnalysis).all()
dict_value_crash_id = [result.crash_id  for result in queryca]
dict_value_testcase_hash = [result.test_case_hash  for result in queryca]
dict_value_crash_type = [result.crash_type  for result in queryca]
dict_value_frames_hash = [result.frames_hash  for result in queryca]
dictca = {"crash_id":dict_value_crash_id, "testcase_hash":dict_value_testcase_hash, "crash_type":dict_value_crash_type, "frames_hash":dict_value_frames_hash}
df_crash_analysis = pd.DataFrame(dictca)
# df_crash_analysis.set_index("crash_id")
df_crash_group = df_crash_analysis.groupby(by="frames_hash")["frames_hash"].count()  #up by frames_hash
print(df_crash_group)
print("*"*30)
print("total number of unique bugs:", df_crash_group.count())

dict_value_benchmark = [benchmark]
dict_value_fuzzer_combination = [fc]
dict_value_experiment_id = [expid]
dict_value_bug_count = [df_crash_group.count()]
df_bug_count = pd.DataFrame({"benchmark":dict_value_benchmark, "fuzzer_combination":dict_value_fuzzer_combination, "experiment_id":dict_value_experiment_id, "bug_count":dict_value_bug_count})
df_bug_count.to_csv(f"csv/fscve_crash_analysis_bugcnt_{benchmark}_{fc}_{expid}.csv")
print(df_bug_count)



frames_hash
01ec3f4a16368aeebe557b7345d03103     2
0212274116b45cb1aefdf592d1ca1149     6
055b07113790287cfcf350b7e708ab8c    16
056c39427a7ea216411e92b2fef07bb2    17
091e2b0acad15effd7329698203f260a     3
                                    ..
f9efba8b88911b424f0576800170ef2f     9
fc70583744155ed14c5a6ddb2d7ef7e1     7
fcba9ae68b702f53d654a32a9d1254aa     1
fd0d1114f3127c61d8d38a41fbc62eed     8
fe6ccbdcd5275066fb3023fbac0625b3     1
Name: frames_hash, Length: 133, dtype: int64
******************************
total number of unique bugs: 133
  benchmark fuzzer_combination  experiment_id  bug_count
0        nm                fca              2        133


In [9]:
from fscve_step1_db import Discovery, Fuzzer, TestCase, FuzzerType, TestCaseType

# query the time of when which fuzzer uses which testcase to find which real bug
querywftcbug =  session.query(Discovery.discovery_id,Discovery.discovery_fuzzer,Fuzzer.fuzzer_name,FuzzerType.description.label("fuzzer_type_description"),Discovery.discovery_time,TestCase.hash.label("testcase_hash"),TestCaseType.description.label("testcase_type_description"),CrashAnalysis.crash_id,CrashAnalysis.crash_type,CrashAnalysis.frames_hash)
querywftcbug = querywftcbug.join((Fuzzer,Discovery.discovery_fuzzer==Fuzzer.fuzzer_id),(FuzzerType,Fuzzer.fuzzer_type_id==FuzzerType.id)).join((TestCase,Discovery.test_case_hash == TestCase.hash),(TestCaseType,TestCase.test_case_type_id == TestCaseType.id))
querywftcbug = querywftcbug.join(CrashAnalysis,Discovery.test_case_hash == CrashAnalysis.test_case_hash)
querywftcbug = querywftcbug.filter(TestCaseType.id==2).order_by(Discovery.discovery_time)

#querywftcbug = querywftcbug.all()
#print(querywftcbug)
#print(querywftcbug.first().keys())
dict_value_discovery_id = [result.discovery_id  for result in querywftcbug]
dict_value_discovery_fuzzer = [result.discovery_fuzzer  for result in querywftcbug]
dict_value_discovery_fuzzer_name = [result.fuzzer_name  for result in querywftcbug]
dict_value_fuzzer_type_description = [result.fuzzer_type_description  for result in querywftcbug]
dict_value_discovery_time = [result.discovery_time  for result in querywftcbug]
dict_value_testcase_hash = [result.testcase_hash  for result in querywftcbug]
dict_value_testcase_type_description = [result.testcase_type_description  for result in querywftcbug]
dict_value_crash_id = [result.crash_id  for result in querywftcbug]
# dict_value_crash_id.sort()
# print(dict_value_crash_id)
dict_value_crash_type = [result.crash_type  for result in querywftcbug]
dict_value_frames_hash = [result.frames_hash  for result in querywftcbug]
dictwftcbug = {"discovery_id":dict_value_discovery_id, "discovery_fuzzer":dict_value_discovery_fuzzer, "discovery_fuzzer_name":dict_value_discovery_fuzzer_name, "fuzzer_type_description":dict_value_fuzzer_type_description, "discovery_time":dict_value_discovery_time, "testcase_hash":dict_value_testcase_hash, "testcase_type_description":dict_value_testcase_type_description,"crash_id":dict_value_crash_id, "crash_type":dict_value_crash_type, "crash_frames_hash":dict_value_frames_hash}
df_wftcbug = pd.DataFrame(dictwftcbug,index=dict_value_crash_id)

df_wftcbug = df_wftcbug.drop_duplicates(['discovery_fuzzer','crash_frames_hash']) #record once for special fuzzer and real bug
df_wftcbug = df_wftcbug.drop_duplicates('crash_id')  #remove duplicate crash_id,which is added to db unreasonably
# print(df_wftcbug)
path_wftcbug_csv = f"csv/fscve_crash_analysis_{benchmark}_{fc}_{expid}.csv"
df_wftcbug.to_csv(path_wftcbug_csv)


In [10]:
# when the fuzzer combination finds the real bug first
find_bug_first_cols_to_keep = ["crash_frames_hash","discovery_time"]
df_find_bug_first = df_wftcbug.drop_duplicates('crash_frames_hash') #record once for special fuzzer and real bug
df_find_bug_first=df_find_bug_first.drop(df_find_bug_first.columns.difference(find_bug_first_cols_to_keep), axis=1)
df_find_bug_first["discovery_time"] = df_find_bug_first["discovery_time"] //60 #minutes
df_find_bug_first=df_find_bug_first.reset_index(drop=True)
print(df_find_bug_first)
df_find_bug_first.to_csv(f"csv/fscve_crash_analysis_bugfindfirsttime_{benchmark}_{fc}_{expid}.csv")

     discovery_time                 crash_frames_hash
0                38  3a8103e52bd56af19a1d1f38820d109a
1                43  c5b7f49d01383031c7e228bfb25b5149
2                43  a12b812cb9bd64247118c3591189e3dd
3                43  edf6719f2ac1abafe779752efd146761
4                51  89190c1ef24ad7d3f7aae8384e9ff6f3
..              ...                               ...
128            1302  7b428cf025116fa1f773e8a98834bb4d
129            1305  40b3ec3613140d6d2673f112c2ec40f6
130            1305  26dd5bcba75a38d74bf3ade2f7047dcf
131            1416  57532014681df0bff73335366858248e
132            1418  389efc1ea8b55022514915dbe6fd820e

[133 rows x 2 columns]


In [11]:
from pyvenn import venn
import matplotlib.pyplot as plt
import seaborn as sns

#obal plot size setting
plt.rcParams['figure.figsize']=(38.4, 28.8)

#visualize the history of the special fuzzer's bug findings
array_count_by_time = []
bugcnt = len(df_wftcbug)

# ValueError: Can only compare identically-labeled Series objects
#df_wftcbug = df_wftcbug.reset_index(drop=True)

#print(df_wftcbug)
#df_wftcbug.to_csv("tmpcrash.csv")
# print(df_wftcbug)
# print("bugcnt:",bugcnt)
# accumulate by time and fuzzer
for i in df_wftcbug['crash_id']:#0529
#for i in range(0,len(df_wftcbug)):
    # print("i:",i,"*"*10,(df_wftcbug['discovery_time'] <= df_wftcbug.loc[i,'discovery_time']))
    # print(len(df_wftcbug[ (df_wftcbug['discovery_time'] <= df_wftcbug.loc[i,'discovery_time']) & (df_wftcbug['discovery_fuzzer_name'] == df_wftcbug.loc[i, 'discovery_fuzzer_name']) ]))
    tmp_time = df_wftcbug.loc[i,'discovery_time']
    print(tmp_time)
    array_count_by_time.append(len(df_wftcbug[ (df_wftcbug['discovery_time'] <= tmp_time) & (df_wftcbug['discovery_fuzzer_name'] == df_wftcbug.loc[i, 'discovery_fuzzer_name']) ]))
# print(array_count_by_time)
# print(dict_value_discovery_time)
# print(dict_value_discovery_fuzzer_name)
# print("*"*30)
# sampling data every ten minutes
df_plot= pd.DataFrame({"bug_count":array_count_by_time, "discovery_time":[t//60 for t in df_wftcbug['discovery_time']], "fuzzer":df_wftcbug['discovery_fuzzer_name']})

fuzzers = df_wftcbug['discovery_fuzzer_name'].unique()
df_top = pd.DataFrame({"bug_count":[0 for fuzzer in fuzzers], "discovery_time":[0 for fuzzer in fuzzers], "fuzzer":df_wftcbug['discovery_fuzzer_name'].unique()})

df_plot = pd.concat([df_top, df_plot]).reset_index(drop = True)

# ValueError: Can only compare identically-labeled Series objects
df_plot.set_index("discovery_time")


print(df_plot)

# plt.figure(figsize=((df_plot.loc[df_plot["discovery_time"].last_valid_index(),"discovery_time"]+10)/60,6))

sns.set(style="whitegrid")
sns.set_palette(sns.color_palette("hls", len(df_wftcbug['discovery_fuzzer_name'].unique())))
sns.lineplot(x="discovery_time", y="bug_count", hue="fuzzer",data=df_plot)
sns.despine(right=True,top=True)

# plt.xticks(ticks=range(0,df_plot.loc[df_plot["discovery_time"].last_valid_index(),"discovery_time"],20),rotation = 45)
plt.yticks(ticks=range(0,max(array_count_by_time)+1,1))
plt.xlabel("Discovery Time (Minutes)")
plt.ylabel("Bug Count")
plt.title("Bug Discovery History",fontsize='xx-large', fontweight='heavy')
plt.savefig(f"image/fuzzer_bug_history_{benchmark}_{fc}_{expid}.svg")
plt.show()


ModuleNotFoundError: No module named 'pyvenn'

In [ ]:
# draw the venn diagram to visualize the unique or common bug findings
dict_bug_tag = {}
bug_tag = 1

def crash_analysis_to_bug_tag(df):
    global dict_bug_tag,bug_tag
    for row in df['crash_id']:
        # print(df.loc[row, "crash_frames_hash"])
        # print(dict_bug_tag.keys())
        if df.loc[row, "crash_frames_hash"]  not in dict_bug_tag.keys():
            dict_bug_tag[df.loc[row, "crash_frames_hash"]] = bug_tag
            bug_tag +=1

crash_analysis_to_bug_tag(df_wftcbug)
fuzzers = df_wftcbug['discovery_fuzzer_name'].unique()
# print(fuzzers)
# print(dict_bug_tag)
dict_fuzzer_to_tags = {}

# get bug tags for every fuzzer
for fuzzer in fuzzers:
    tags = []
    df_tmp = df_wftcbug[df_wftcbug['discovery_fuzzer_name'] == fuzzer]
    for row in df_tmp['crash_id']:
        tag = dict_bug_tag[df_tmp.loc[row, "crash_frames_hash"]]
        if tag not in tags:
            tags.append(tag)
    dict_fuzzer_to_tags[fuzzer] = tags

# limit fuzzer numbers to 6 for venn graphs
if len(fuzzers)>6:
    fuzzers=fuzzers[:6]

fuzzer_count = len(fuzzers)
#print(dict_fuzzer_to_tags)
venn_functions = [venn.venn2,venn.venn3,venn.venn4,venn.venn5,venn.venn6]
labels = venn.get_labels([dict_fuzzer_to_tags[fuzzer] for fuzzer in fuzzers], fill=['number', 'percent'])
func = venn_functions[fuzzer_count-2]
fig, ax = func(labels, names=fuzzers)
fig.savefig(f'image/fuzzer_bug_venn_{benchmark}_{fc}_{expid}.svg', bbox_inches='tight')

In [ ]:
from matplotlib.patches import Rectangle
# function:which fuzzer finds the special bug earlist?
# visualize when the real bug is found
from fscve_step1_db import Discovery, Fuzzer, TestCase, FuzzerType, TestCaseType

def autolabel(rects, texts):
    __i__ = 0
    for rect in rects:
        height=rect.get_height()
        plt.text(rect.get_x(), 1.05 * height, "{}\n{}".format(int(height),texts[__i__]),rotation = 90)
        __i__ += 1


# query the time of when which fuzzer uses which testcase to find which real bug
querywftcbug =  session.query(Discovery.discovery_id,Discovery.discovery_fuzzer,Fuzzer.fuzzer_name,FuzzerType.description.label("fuzzer_type_description"),Discovery.discovery_time,TestCase.hash.label("testcase_hash"),TestCaseType.description.label("testcase_type_description"),CrashAnalysis.crash_id,CrashAnalysis.crash_type,CrashAnalysis.frames_hash)
querywftcbug = querywftcbug.join((Fuzzer,Discovery.discovery_fuzzer==Fuzzer.fuzzer_id),(FuzzerType,Fuzzer.fuzzer_type_id==FuzzerType.id)).join((TestCase,Discovery.test_case_hash == TestCase.hash),(TestCaseType,TestCase.test_case_type_id == TestCaseType.id))
querywftcbug = querywftcbug.join(CrashAnalysis,Discovery.test_case_hash == CrashAnalysis.test_case_hash)
querywftcbug = querywftcbug.filter(TestCaseType.id==2).order_by(Discovery.discovery_time)

#querywftcbug = querywftcbug.all()
#print(querywftcbug)
#print(querywftcbug.first().keys())
dict_value_discovery_id = [result.discovery_id  for result in querywftcbug]
dict_value_discovery_fuzzer = [result.discovery_fuzzer  for result in querywftcbug]
dict_value_discovery_fuzzer_name = [result.fuzzer_name  for result in querywftcbug]
dict_value_fuzzer_type_description = [result.fuzzer_type_description  for result in querywftcbug]
dict_value_discovery_time = [result.discovery_time  for result in querywftcbug]
dict_value_testcase_hash = [result.testcase_hash  for result in querywftcbug]
dict_value_testcase_type_description = [result.testcase_type_description  for result in querywftcbug]
dict_value_crash_id = [result.crash_id  for result in querywftcbug]
dict_value_crash_type = [result.crash_type  for result in querywftcbug]
dict_value_frames_hash = [result.frames_hash  for result in querywftcbug]
dictwftcbug = {"discovery_id":dict_value_discovery_id, "discovery_fuzzer":dict_value_discovery_fuzzer, "discovery_fuzzer_name":dict_value_discovery_fuzzer_name, "fuzzer_type_description":dict_value_fuzzer_type_description, "discovery_time":dict_value_discovery_time, "testcase_hash":dict_value_testcase_hash, "testcase_type_description":dict_value_testcase_type_description,"crash_id":dict_value_crash_id, "crash_type":dict_value_crash_type, "crash_frames_hash":dict_value_frames_hash}
df_wftcbug = pd.DataFrame(dictwftcbug,index=dict_value_crash_id)
# df_wftcbug.set_index("crash_id")
df_wftcbug = df_wftcbug.drop_duplicates(['crash_frames_hash']) #record once for special fuzzer and real bug
# print(df_wftcbug)
df_plot= pd.DataFrame({"crash_id":["bug{}".format(cid) for cid in df_wftcbug["crash_id"]], "discovery_time":[t//60 for t in df_wftcbug['discovery_time']], "fuzzer":df_wftcbug['discovery_fuzzer_name']})
print(df_plot)
path_wftcbug_csv = f"csv/fscve_crash_earlist_{benchmark}_{fc}_{expid}.csv"

df_plot.to_csv(path_wftcbug_csv)

sns.set(style="whitegrid")
bar = sns.barplot(x="crash_id", y='discovery_time',data=df_plot,yerr=0.001)
sns.despine()
plt.title("The Time to Find The Special Bug")
plt.ylabel("Discovery Time (Minutes)")
plt.xlabel("Crash Id")
plt.yticks(range(0,df_plot.loc[df_plot.last_valid_index(),'discovery_time']+100,50))
plt.xticks(rotation = 45)
hatches = ['\\', '+', 'x', '\\', '*', '-']
for i,thisbar in enumerate(bar.patches):
    print(thisbar)
    # Set a different hatch for each bar
    thisbar.set_hatch(hatches[i % len(hatches)])

#to specify which fuzzer find the bug earlist
autolabel([thisbar for i,thisbar in enumerate(bar.patches)], df_plot["fuzzer"].to_list())
print(f"image/fuzzer_bug_earlist_{benchmark}_{fc}_{expid}.svg")
plt.savefig(f"image/fuzzer_bug_earlist_{benchmark}_{fc}_{expid}.svg")
plt.show()